# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import re
import pickle

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn import metrics
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV

In [2]:
nltk.download(['wordnet', 'punkt', 'stopwords'])

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
# load data from database
engine = create_engine('sqlite:///disaster_ETL.db')
df = pd.read_sql_table("disaster_ETL", engine)
X = df['message']
Y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    
    """
    Function: split text into words and return the root form of the words
    Args:
      text(str): the message
    Return:
      lemm(list of str): a list of the root form of the message words
    """
    
    # Convert everything to lower case to Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenize text
    words = word_tokenize(text)
    
    # Stop words removal
    stopword = stopwords.words("english")
    
    # Lemmatization
    lemmed = [WordNetLemmatizer().lemmatize(w) for w in words if w not in stopword]
    
    return lemmed

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
# Creating 2 pipelines for 2 different ML models to evaluate they performance

# Pipeline #1 - RandomForestClassifier

pipeline_rf = Pipeline([
    ('vect',CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf',MultiOutputClassifier(estimator=RandomForestClassifier()))
    ])

# Pipeline #2 - MultinomialNB
pipeline_mnb = Pipeline([
    ('vect',CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf',MultiOutputClassifier(estimator=MultinomialNB()))
    ])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [7]:
# Train Random Forest pipeline
pipeline_rf.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [8]:
# Train Multinomial NB pipeline
pipeline_mnb.fit(X=_train,y_train)

/opt/conda/lib/python3.6/site-packages/sklearn/naive_bayes.py:461: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ssifier(estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
# To print the classification report for each 'Y' label

def classification_reports(y_test, y_pred):    

    i = 0
    for col in y_test:
        print('label {} : {}'.format(i+1, col))
        print(metrics.classification_report(y_test[col], y_pred[:, i]))
        i = i + 1
    accuracy = (y_pred == y_test.values).mean()
    print('The model accuracy is {:.2f}'.format(accuracy))

In [10]:
# Test #1: RandomForestClassifier

y_pred_rf = pipeline_rf.predict(X_test)
classification_reports(y_test,y_pred_rf)

label 1 : related
             precision    recall  f1-score   support

          0       0.61      0.48      0.53      1534
          1       0.85      0.90      0.88      4972
          2       0.24      0.38      0.30        48

avg / total       0.79      0.80      0.79      6554

label 2 : request
             precision    recall  f1-score   support

          0       0.88      0.98      0.93      5379
          1       0.83      0.39      0.53      1175

avg / total       0.87      0.88      0.86      6554

label 3 : offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6525
          1       0.00      0.00      0.00        29

avg / total       0.99      1.00      0.99      6554

label 4 : aid_related
             precision    recall  f1-score   support

          0       0.75      0.86      0.80      3795
          1       0.75      0.60      0.67      2759

avg / total       0.75      0.75      0.74      6554

label 5 : med

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [11]:
# Test #2: MultinomialNB

y_pred_mnb = pipeline_mnb.predict(X_test)
classification_reports(y_test,y_pred_mnb)

label 1 : related
             precision    recall  f1-score   support

          0       0.78      0.10      0.18      1534
          1       0.78      0.99      0.87      4972
          2       0.00      0.00      0.00        48

avg / total       0.77      0.78      0.70      6554

label 2 : request
             precision    recall  f1-score   support

          0       0.85      0.99      0.92      5379
          1       0.87      0.19      0.31      1175

avg / total       0.85      0.85      0.81      6554

label 3 : offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6525
          1       0.00      0.00      0.00        29

avg / total       0.99      1.00      0.99      6554

label 4 : aid_related
             precision    recall  f1-score   support

          0       0.76      0.85      0.80      3795
          1       0.76      0.63      0.69      2759

avg / total       0.76      0.76      0.76      6554

label 5 : med

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
parameters = {
    'vect__min_df': [1, 5],
    'tfidf__use_idf':[True, False],
    'clf__estimator__n_estimators': [10, 20]
}
cv = GridSearchCV(pipeline_rf,param_grid=parameters,verbose=2)

In [13]:
Tuned_model = cv.fit(X_train,y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, total=  51.4s
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.1min remaining:    0.0s


[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, total=  51.3s
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, total=  50.6s
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5 
[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5, total=  39.2s
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5 
[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5, total=  39.7s
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5 
[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5, total=  39.1s
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=1 
[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=1, total=  52.7s
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=1 
[CV]  c

[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed: 29.5min finished


In [14]:
Tuned_model.best_score_

0.26238429457837453

In [15]:
Tuned_model.best_params_

{'clf__estimator__n_estimators': 20,
 'tfidf__use_idf': False,
 'vect__min_df': 5}

In [16]:
### We can also try RanomizedSearchCV
### Useful when computation power is low

parameters = {
    'vect__min_df': [1, 5],
    'tfidf__use_idf':[True, False],
    'clf__estimator__n_estimators': [10, 20],
    'clf__estimator__min_samples_split':[2, 5, 10]
}
rs = RandomizedSearchCV(pipeline_rf,parameters,cv=5,return_train_score=False,n_iter=3)

In [ ]:
## rs.fit(X_train,y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [17]:
tuned_clf = Tuned_model.predict(X_test)

In [18]:
classification_reports(y_test, tuned_clf)

label 1 : related
             precision    recall  f1-score   support

          0       0.65      0.47      0.54      1534
          1       0.85      0.92      0.88      4972
          2       0.30      0.35      0.33        48

avg / total       0.80      0.81      0.80      6554

label 2 : request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      5379
          1       0.85      0.44      0.58      1175

avg / total       0.88      0.89      0.87      6554

label 3 : offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6525
          1       1.00      0.03      0.07        29

avg / total       1.00      1.00      0.99      6554

label 4 : aid_related
             precision    recall  f1-score   support

          0       0.78      0.84      0.81      3795
          1       0.75      0.67      0.71      2759

avg / total       0.77      0.77      0.77      6554

label 5 : med

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

#### Multinomial NAIVE BAYES and RANDOM FOREST gave similar results.

Also try 


 SVM (time exhaustive)
 
 AdaBoost (poor performance)

### 9. Export your model as a pickle file

In [19]:
# Create a pickle file for the model
with open ('disaster_model.pkl', 'wb') as file:
    pickle.dump(Tuned_model, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.